# A Mood Dashboard for the MAYke It! Hackathon

## Metrics
- Hours of Sleep per Night
- Minutes of Workout (Physical Activity)
- Time Spent Socializing in Minutes
- Energy Levels: On a scale from 1-10
- Mood: ten categories: "happy", "sad", "anxious", "depressed", "content", "angry", "excited", "bored", "stressed", "relaxed"
- Screen Time in Minutes
- Sleep Quality on a scale from 1-10
- Rainfall in Millimeters
- Sunshine in Hours

## Generating a synthetic dataset for these metrics

In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [2]:
moods = ["happy", "sad", "anxious", "depressed", "content", 
         "angry", "excited", "bored", "stressed", "relaxed"]

In [3]:
def generate_synthetic_data(num_days):
    data = []
    current_date = datetime.now()
    good_moods = ["happy", "content", "excited", "relaxed"]
    bad_moods = ["sad", "depressed", "bored", "anxious", "stressed", "angry"]
    mixed_moods = ["content", "relaxed", "bored"]

    for i in range(num_days):
        date = current_date - timedelta(days=i)
        mood = random.choice(good_moods + bad_moods)

        if mood in good_moods:
            sleep_hours = round(random.uniform(6, 9), 2)
            workout_minutes = random.randint(30, 120)
            social_interaction_minutes = random.randint(60, 300)
            energy_level = random.randint(7, 10)
        else:
            sleep_hours = round(random.uniform(4, 7), 2)
            workout_minutes = random.randint(0, 60)
            social_interaction_minutes = random.randint(0, 120)
            energy_level = random.randint(1, 6)

        if mood in mixed_moods:
            sleep_quality = random.randint(7, 10)
        else:
            sleep_quality = random.randint(1, 6)

        if mood in bad_moods:
            screen_time_minutes = random.randint(240, 480)
        else:
            screen_time_minutes = random.randint(60, 240)

        rainfall_mm = round(random.uniform(0, 100), 2)
        sunshine_hours = round(random.uniform(0, 12), 2)

        data.append((date, sleep_hours, workout_minutes, social_interaction_minutes, energy_level, mood, screen_time_minutes, sleep_quality, rainfall_mm, sunshine_hours))
    
    return pd.DataFrame(data, columns=['date', 'sleep_hours', 'workout_minutes', 'social_interaction_minutes', 'energy_level', 'mood', 'screen_time_minutes', 'sleep_quality', 'rainfall_mm', 'sunshine_hours'])

In [4]:
num_days = 1095 # three years
synthetic_data = generate_synthetic_data(num_days)
print(synthetic_data.head())

                        date  sleep_hours  workout_minutes  \
0 2024-05-18 11:25:25.862667         4.54                2   
1 2024-05-17 11:25:25.862667         5.87               18   
2 2024-05-16 11:25:25.862667         5.71               21   
3 2024-05-15 11:25:25.862667         8.14               70   
4 2024-05-14 11:25:25.862667         6.02               51   

   social_interaction_minutes  energy_level       mood  screen_time_minutes  \
0                          69             1        sad                  276   
1                          92             5  depressed                  471   
2                         111             2        sad                  472   
3                         100            10    excited                  186   
4                          59             6      bored                  388   

   sleep_quality  rainfall_mm  sunshine_hours  
0              3        95.57            7.83  
1              3        63.22            4.22  
2       

In [5]:
synthetic_data.to_csv("data/synthetic_mood_data.csv", index=False)

In [6]:
!pip install psycopg2
!pip install python-dotenv

In [7]:
import os
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv

# Load environment variables from .env file. Set your database connection data in this file!
load_dotenv("postgres.env")

# Retrieve environment variables
dbname = os.getenv('PGDATABASE')
user = os.getenv('PGUSER')
password = os.getenv('PGPASSWORD')
host = os.getenv('PGHOST')
port = os.getenv('PGPORT')

In [8]:
def reset_table():
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cursor = conn.cursor()
    
    # Drop the existing table
    cursor.execute("DROP TABLE IF EXISTS user_activities")
    
    # Create the table
    cursor.execute("""
    CREATE TABLE user_activities (
        id SERIAL PRIMARY KEY,
        date DATE,
        sleep_hours FLOAT,
        workout_minutes INT,
        social_interaction_minutes INT,
        energy_level INT,
        mood VARCHAR(50),
        screen_time_minutes INT,
        sleep_quality INT,
        rainfall_mm FLOAT,
        sunshine_hours FLOAT
    )
    """)
    
    conn.commit()
    cursor.close()
    conn.close()

In [9]:
def insert_data_to_postgresql(dataframe):
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        cursor = conn.cursor()
        
        # Insert data
        for _, row in dataframe.iterrows():
            cursor.execute(
                sql.SQL("""
                    INSERT INTO user_activities (
                        date, sleep_hours, workout_minutes, social_interaction_minutes,
                        energy_level, mood, screen_time_minutes, sleep_quality,
                        rainfall_mm, sunshine_hours
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """),
                (
                    row['date'], row['sleep_hours'], row['workout_minutes'],
                    row['social_interaction_minutes'], row['energy_level'],
                    row['mood'], row['screen_time_minutes'], row['sleep_quality'],
                    row['rainfall_mm'], row['sunshine_hours']
                )
            )
        
        # Commit changes and close the connection
        conn.commit()
        cursor.close()
        conn.close()
        
        print("Data inserted successfully!")
    
    except Exception as error:
        print(f"Error: {error}")

insert_data_to_postgresql(synthetic_data)

Data inserted successfully!


In [10]:
def verify_data():
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        cursor = conn.cursor()
        
        # Query to count rows in the table
        cursor.execute("SELECT COUNT(*) FROM user_activities;")
        count = cursor.fetchone()[0]
        
        # Fetch some sample rows
        cursor.execute("SELECT * FROM user_activities LIMIT 5;")
        rows = cursor.fetchall()
        
        cursor.close()
        conn.close()
        
        print(f"Total rows inserted: {count}")
        print("Sample rows:")
        for row in rows:
            print(row)
    
    except Exception as error:
        print(f"Error: {error}")

# Verify data insertion
verify_data()

Total rows inserted: 9855
Sample rows:
(1, datetime.date(2021, 5, 18), 8.26, 51, 71, 9, 'sad', 459, 7, 58.21, 2.87)
(2, datetime.date(2021, 5, 19), 4.15, 48, 98, 4, 'sad', 326, 1, 7.73, 7.16)
(3, datetime.date(2021, 5, 20), 5.7, 16, 288, 7, 'bored', 291, 5, 5.3, 2.38)
(4, datetime.date(2021, 5, 21), 7.51, 77, 246, 7, 'stressed', 238, 2, 3.18, 0.69)
(5, datetime.date(2021, 5, 22), 7.16, 29, 31, 10, 'content', 195, 4, 30.33, 2.92)


## Building the dashboard

In [11]:
!pip install dash cloudpickle fsspec partd pyyaml toolz locket msgpack sortedcontainers tblib zict

In [12]:
#!pip uninstall -y typing_extensions sqlalchemy
!pip install typing_extensions==4.5.0 sqlalchemy==1.4.35

In [13]:
!pip install dash-bootstrap-components

In [14]:
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
import random
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import webbrowser

# Load environment variables from .env file
load_dotenv('postgres.env')

# Retrieve environment variables
dbname = os.getenv('PGDATABASE')
user = os.getenv('PGUSER')
password = os.getenv('PGPASSWORD')
host = os.getenv('PGHOST')
port = os.getenv('PGPORT')

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

# Fetch data from PostgreSQL
def fetch_data():
    query = "SELECT * FROM user_activities;"
    df = pd.read_sql(query, engine)
    
    # Ensure date is datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter data to last 30 days
    last_30_days = datetime.now() - timedelta(days=30)
    df = df[df['date'] >= last_30_days]
    
    return df

# Load data
df = fetch_data()

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Mood Dashboard", style={'text-align': 'center'}),
    html.Img(src="https://images.unsplash.com/photo-1611267254323-4db7b39c732c", style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "200px"}),
    html.H3("Come on, let's get something done. I'll help you!", style={"text-align": "center", "margin-top": "20px"}),
    html.Div([
        dcc.Interval(id='interval-component', interval=1000, n_intervals=0),
        html.H2(id='timer', children='25:00'),
        dbc.Button('Start', id='start-button', n_clicks=0, color='primary', style={'margin': '5px'}),
        dbc.Button('Stop', id='stop-button', n_clicks=0, color='danger', style={'margin': '5px'}),
        dbc.Button('Reset', id='reset-button', n_clicks=0, color='secondary', style={'margin': '5px'}),
    ], style={'text-align': 'center', 'padding': '20px'}),
    html.Div([
        dcc.Graph(id='mood-bar-chart'),
        dcc.Graph(id='sleep-energy-scatter-plot'),
    ], style={'display': 'flex'}),
    html.Div([
        dcc.Graph(id='sleep-quality-mood-chart'),
        dcc.Graph(id='activity-screen-time-mood-chart'),
    ], style={'display': 'flex'}),
    html.Div([
        dcc.Graph(id='social-interaction-line-chart'),
        dcc.Graph(id='weather-mood-scatter-plot'),
    ], style={'display': 'flex'}),
    html.Div(id='summary-stats')
])

@app.callback(
    Output('mood-bar-chart', 'figure'),
    Input('mood-bar-chart', 'id')
)
def update_mood_bar_chart(_):
    # Aggregate mood counts
    mood_counts = df['mood'].value_counts().reset_index()
    mood_counts.columns = ['mood', 'count']
    fig = px.bar(mood_counts, x='mood', y='count', title='Mood Counts Over Last 30 Days')
    return fig

@app.callback(
    Output('sleep-energy-scatter-plot', 'figure'),
    Input('sleep-energy-scatter-plot', 'id')
)
def update_sleep_energy_scatter_plot(_):
    fig = px.scatter(df, x='sleep_hours', y='energy_level', trendline='ols', title='Sleep Hours vs. Energy Level')
    return fig

@app.callback(
    Output('sleep-quality-mood-chart', 'figure'),
    Input('sleep-quality-mood-chart', 'id')
)
def update_sleep_quality_mood_chart(_):
    fig = px.box(df, x='mood', y='sleep_quality', title='Sleep Quality vs. Mood')
    return fig

@app.callback(
    Output('activity-screen-time-mood-chart', 'figure'),
    Input('activity-screen-time-mood-chart', 'id')
)
def update_activity_screen_time_mood_chart(_):
    # Aggregate workout minutes and screen time minutes by mood
    agg_activity = df.groupby('mood', as_index=False)['workout_minutes'].mean(numeric_only=True)
    agg_screen_time = df.groupby('mood', as_index=False)['screen_time_minutes'].mean(numeric_only=True)

    # Combine the data
    agg_combined = pd.merge(agg_activity, agg_screen_time, on='mood')
    fig = px.line(agg_combined, x='mood', y=['workout_minutes', 'screen_time_minutes'], title='Average Physical Activity and Screen Time vs. Mood')
    fig.update_traces(line=dict(color='red'), selector=dict(name='workout_minutes'))
    fig.update_traces(line=dict(color='blue'), selector=dict(name='screen_time_minutes'))
    return fig

@app.callback(
    Output('social-interaction-line-chart', 'figure'),
    Input('social-interaction-line-chart', 'id')
)
def update_social_interaction_line_chart(_):
    df_grouped = df.groupby('date').mean(numeric_only=True).reset_index()
    fig = px.line(df_grouped, x='date', y='social_interaction_minutes', title='Average Social Interaction Minutes Over Last 30 Days')
    return fig

@app.callback(
    Output('weather-mood-scatter-plot', 'figure'),
    Input('weather-mood-scatter-plot', 'id')
)
def update_weather_mood_scatter_plot(_):
    fig = px.scatter(df, x='rainfall_mm', y='sunshine_hours', color='mood', title='Effects of Rainfall and Sunshine Hours on Mood')
    return fig

@app.callback(
    Output('interval-component', 'disabled'),
    [Input('start-button', 'n_clicks'),
     Input('stop-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')]
)
def set_interval_disabled(start_clicks, stop_clicks, reset_clicks):
    # Disable the interval component when the timer is stopped or reset
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'stop-button' in changed_id or 'reset-button' in changed_id:
        return True
    else:
        return False

@app.callback(
    Output('timer', 'children'),
    [Input('interval-component', 'n_intervals'),
     Input('start-button', 'n_clicks'),
     Input('stop-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')]
)
def update_timer(n_intervals, start_clicks, stop_clicks, reset_clicks):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'start-button' in changed_id:
        return '25:00'
    elif 'stop-button' in changed_id:
        return dash.no_update
    elif 'reset-button' in changed_id:
        return '25:00'
    else:
        minutes = (25 - (n_intervals // 60)) % 60
        seconds = 59 - (n_intervals % 60)
        return '{:02d}:{:02d}'.format(minutes, seconds)

# Run the app on a different port and open in a new tab
if __name__ == '__main__':
    port = 8051
    webbrowser.open_new_tab(f"http://127.0.0.1:{port}")
    app.run_server(debug=True, port=port)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[14], line 123, in update_social_interaction_line_chart(_='social-interaction-line-chart')
    117 @app.callback(
    118     Output('social-interaction-line-chart', 'figure'),
    119     Input('social-interaction-line-chart', 'id')
    120 )
    121 def update_social_interaction_line_chart(_):
    122     df_grouped = df.groupby('date').mean(numeric_only=True).reset_index()
--> 123     fig = px.line(df_grouped, x='date', y='social_interaction_minutes', title='Average Social Interaction Minutes Over Last 30 Days')
        df_grouped =          date           id  sleep_hours  workout_minutes  \
0  2024-04-19  4871.111111     5.835556        42.555556   
1  2024-04-20  4871.000000     6.710000        55.666667   
2  2024-04-21  4870.888889     7.656667        52.555556   
3  2024-04-22  4870.777778     5.851111        59.888889   